In [1]:
import csv
import random
import nltk 
from nltk.tokenize import word_tokenize
import datetime
import time
import random
import itertools
import numpy as np
import pickle
from numpy.linalg import cholesky


In [25]:
def newsample(nnn,ratio):
    if ratio >len(nnn):
        return random.sample(nnn*(ratio//len(nnn)+1),ratio)
    else:
        return random.sample(nnn,ratio)

In [3]:
def preprocess_user_file(file='ClickData4.tsv',npratio=4):
    userid_dict={}
    with open(file) as f:
        userdata=f.readlines()
    for user in userdata:
        line=user.strip().split('\t')
        userid=line[0]
        # map user_id to index
        if userid not in userid_dict:
            userid_dict[userid]=len(userid_dict)
    
    all_train_id=[]
    all_train_pn=[]    
    all_label=[]
    
    all_test_id=[]
    all_test_pn=[]    
    all_test_label=[]
    all_test_index=[]
    
    all_user_pos=[]
    all_test_user_pos=[]
    
    for user in userdata:
        line=user.strip().split('\t')
        userid=line[0]
        # split userdata in impressions
        if len(line)==4:
            impre=[x.split('#TAB#') for x in line[2].split('#N#')]
        if len(line)==3:
            impre=[x.split('#TAB#') for x in line[2].split('#N#')]
        else:
            raise NotImplementedError() # HB edit
    
        trainpos=[x[0].split() for x in impre]
        trainneg=[x[1].split() for x in impre]
         
        poslist=list(itertools.chain(*(trainpos)))
        neglist=list(itertools.chain(*(trainneg)))
    
        
        if len(line)==4:
            testimpre=[x.split('#TAB#') for x in line[3].split('#N#')]
            testpos=[x[0].split() for x in testimpre]
            testneg=[x[1].split() for x in testimpre]
            
            
            for i in range(len(testpos)):
                sess_index=[]
                sess_index.append(len(all_test_pn))
                posset=list(set(poslist))
                allpos=[int(p) for p in random.sample(posset,min(50,len(posset)))[:50]]
                allpos+=[0]*(50-len(allpos))
        
                
                for j in testpos[i]:
                    all_test_pn.append(int(j))
                    all_test_label.append(1)
                    all_test_id.append(userid_dict[userid])
                    all_test_user_pos.append(allpos)
                    
                for j in testneg[i]:
                    all_test_pn.append(int(j))
                    all_test_label.append(0)
                    all_test_id.append(userid_dict[userid])
                    all_test_user_pos.append(allpos)
                sess_index.append(len(all_test_pn))
                all_test_index.append(sess_index)
                
    
    
                
        for impre_id in range(len(trainpos)):
            for pos_sample in trainpos[impre_id]:
    
                pos_neg_sample=newsample(trainneg[impre_id],npratio)
                pos_neg_sample.append(pos_sample)
                temp_label=[0]*npratio+[1]
                temp_id=list(range(npratio+1))
                random.shuffle(temp_id)
    
                
                shuffle_sample=[]
                shuffle_label=[]
                for id in temp_id:
                    shuffle_sample.append(int(pos_neg_sample[id]))
                    shuffle_label.append(temp_label[id])
                
                posset=list(set(poslist)-set([pos_sample]))
                allpos=[int(p) for p in random.sample(posset,min(50,len(posset)))[:50]]
                allpos+=[0]*(50-len(allpos))
                all_train_pn.append(shuffle_sample)
                all_label.append(shuffle_label)
                all_train_id.append(userid_dict[userid])
                all_user_pos.append(allpos)
            
    all_train_pn=np.array(all_train_pn,dtype='int32')
    all_label=np.array(all_label,dtype='int32')
    all_train_id=np.array(all_train_id,dtype='int32')
    all_test_pn=np.array(all_test_pn,dtype='int32')
    all_test_label=np.array(all_test_label,dtype='int32')
    all_test_id=np.array(all_test_id,dtype='int32')
    all_user_pos=np.array(all_user_pos,dtype='int32')
    all_test_user_pos=np.array(all_test_user_pos,dtype='int32')
    return userid_dict,all_train_pn,all_label,all_train_id,all_test_pn,all_test_label,all_test_id,all_user_pos,all_test_user_pos,all_test_index

In [ ]:
DEBUG = True




In [24]:
def preprocess_news_file(file='DocMeta3.tsv'):
    with open(file) as f:
        newsdata=f.readlines()
    
    news={}
    for newsline in newsdata:
        line=newsline.strip().split('\t')
        news[line[1]]=[line[2],line[3],word_tokenize(line[6].lower())]
    word_dict_raw={'PADDING':[0,999999]} # key: 'word', value: [index, counts]
    
    for docid in news:
        for word in news[docid][2]:
            if word in word_dict_raw:
                word_dict_raw[word][1]+=1
            else:
                word_dict_raw[word]=[len(word_dict_raw),1]
    word_dict={}
    for i in word_dict_raw:
        # only include word with counter > 2
        if word_dict_raw[i][1]>=2:
            word_dict[i]=[len(word_dict),word_dict_raw[i][1]]
    print("Vocab: {}  Raw: {}".format(len(word_dict),len(word_dict_raw)))
    
    news_words=[[0]*30] # max_len_news_title = 30
    news_index={'0':0} # dictionary news indices
    for newsid in news:
        word_id=[]
        news_index[newsid]=len(news_index)
        # get word_ids from news title
        for word in news[newsid][2]:
            if word in word_dict:
                word_id.append(word_dict[word][0])
        
        #pad sequence
        word_id=word_id[:30] # max_len_news_title
        news_words.append(word_id+[0]*(30-len(word_id))) # note: 0 as padding value
        
    news_words=np.array(news_words,dtype='int32') 
    return word_dict,news_words,news_index

In [5]:
def get_embedding(word_dict):
    embedding_dict={}
    
    # for each word in vocabulary, look up and store glove embedding vector
    # words from dictionary that don't appear in Glove are discarded (?)
    cnt=0
    with open('/data/wuch/glove.840B.300d.txt','rb')as f:
        linenb=0
        while True:
            line=f.readline()
            if len(line)==0:
                break
            line = line.split()
            word=line[0].decode()
            linenb+=1
            if len(word) != 0:
                vec=[float(x) for x in line[1:]]
                # only store words that appear in dictionary
                if word in word_dict:
                    embedding_dict[word]=vec
                    if cnt%1000==0:
                        print(cnt,linenb,word)
                    cnt+=1
    
    embedding_matrix=[0]*len(word_dict)
    cand=[]
    # convert embedding dict into matrix
    # pretrained values are floats
    # 
    for i in embedding_dict:
        embedding_matrix[word_dict[i][0]]=np.array(embedding_dict[i],dtype='float32')
        cand.append(embedding_matrix[word_dict[i][0]])
    cand=np.array(cand,dtype='float32')
    mu=np.mean(cand, axis=0)
    Sigma=np.cov(cand.T)
    norm=np.random.multivariate_normal(mu, Sigma, 1)
    for i in range(len(embedding_matrix)):
        # if embedding is int, i.e. no pretrained value, initialise with values from normal distribution
        if type(embedding_matrix[i])==int:
            embedding_matrix[i]=np.reshape(norm, 300)
    embedding_matrix[0]=np.zeros(300,dtype='float32') # placeholder with zero values (?)
    embedding_matrix=np.array(embedding_matrix,dtype='float32')
    print(embedding_matrix.shape)
    return embedding_matrix

In [6]:
userid_dict, all_train_pn, all_label, all_train_id, \
    all_test_pn, all_test_label, all_test_id, \
    all_user_pos, all_test_user_pos, all_test_index \
        = preprocess_user_file(file='./ClickData_sample.tsv')

In [7]:
len(userid_dict.keys())

50

In [8]:
len(all_train_pn)

1291

In [9]:
all_train_pn[:4]

array([[10501, 29467, 13735, 31165, 26836],
       [22212, 13735, 15719, 36713,  4450],
       [38400, 39980, 27040, 17229, 18817],
       [12238, 28796,  2083, 21200, 30403]], dtype=int32)

In [10]:
all_label[:4]

array([[0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0],
       [0, 0, 0, 1, 0]], dtype=int32)

In [11]:
all_train_id[:20]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
      dtype=int32)

In [12]:
len(all_train_id)

1291

In [13]:
len(all_test_id)

7363

In [14]:
word_dict, news_words, news_index = preprocess_news_file(file='./DocMeta_sample.tsv')

56 388


In [21]:
word_dict.items()

dict_items([('PADDING', [0, 999999]), ('the', [1, 10]), ('2018', [2, 4]), ('?', [3, 2]), ('are', [4, 3]), ('15', [5, 2]), ('for', [6, 12]), ('13', [7, 3]), ('tips', [8, 2]), ('that', [9, 6]), ('can', [10, 2]), ('help', [11, 2]), ('you', [12, 4]), ('your', [13, 3]), ('at', [14, 4]), ('dainese', [15, 2]), ("'s", [16, 9]), ('vision', [17, 2]), ('of', [18, 10]), ('a', [19, 6]), ('and', [20, 7]), ('best', [21, 5]), ('in', [22, 6]), ('work', [23, 3]), ('will', [24, 2]), ('return', [25, 2]), (',', [26, 10]), ('with', [27, 7]), (':', [28, 5]), ("'", [29, 6]), ('top', [30, 2]), ('games', [31, 3]), ('have', [32, 3]), ('all', [33, 3]), ('over', [34, 3]), ('get', [35, 2]), ('from', [36, 3]), ('out', [37, 2]), ('on', [38, 3]), ('could', [39, 2]), ('new', [40, 2]), ('tom', [41, 2]), ('is', [42, 3]), ('list', [43, 2]), ('to', [44, 11]), ('priest', [45, 2]), ('how', [46, 2]), ('his', [47, 2]), ('says', [48, 2]), ('$', [49, 2]), ('stores', [50, 2]), ('cities', [51, 2]), ('1', [52, 2]), ('go', [53, 2]),

In [23]:
news_words.shape

(51, 30)

In [15]:
embedding_mat=get_embedding(word_dict)

FileNotFoundError: [Errno 2] No such file or directory: '/data/wuch/glove.840B.300d.txt'

In [16]:

def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best


def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)


In [17]:

def generate_batch_data_train(all_train_pn,all_label,all_train_id,batch_size):
    inputid = np.arange(len(all_label))
    np.random.shuffle(inputid)
    y=all_label
    batches = [inputid[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            candidate = news_words[all_train_pn[i]]
            candidate_split=[candidate[:,k,:] for k in range(candidate.shape[1])]
            browsed_news=news_words[all_user_pos[i]]
            browsed_news_split=[browsed_news[:,k,:] for k in range(browsed_news.shape[1])]
            userid=np.expand_dims(all_train_id[i],axis=1)
            label=all_label[i]
            yield (candidate_split +browsed_news_split+[userid], label)

In [18]:
def generate_batch_data_test(all_test_pn,all_label,all_test_id,batch_size):
    inputid = np.arange(len(all_label))
    y=all_label
    batches = [inputid[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]

    while (True):
        for i in batches:
            candidate = news_words[all_test_pn[i]]
            browsed_news=news_words[all_test_user_pos[i]]
            browsed_news_split=[browsed_news[:,k,:] for k in range(browsed_news.shape[1])]
            userid=np.expand_dims(all_test_id[i],axis=1)
            label=all_label[i]

            yield ([candidate]+ browsed_news_split+[userid], label)
    

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import keras
from keras.layers import *
from keras.models import Model
from keras import backend as K
from keras.optimizers import *
npratio=4
results=[]
    
    
MAX_SENT_LENGTH=30
MAX_SENTS=50

##user embedding - word & article level
user_id = Input(shape=(1,), dtype='int32')
user_embedding_layer= Embedding(len(userid_dict), 50,trainable=True)
user_embedding= user_embedding_layer(user_id)
user_embedding_word= Dense(200,activation='relu')(user_embedding)
user_embedding_word= Flatten()(user_embedding_word)
user_embedding_news= Dense(200,activation='relu')(user_embedding)
user_embedding_news= Flatten()(user_embedding_news)

##news encoder
news_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
embedding_layer = Embedding(len(word_dict) , 300, weights=[embedding_mat],trainable=True)
embedded_sequences = embedding_layer(news_input)
embedded_sequences =Dropout(0.2)(embedded_sequences)

cnnouput = Convolution1D(nb_filter=400, filter_length=3,  padding='same', activation='relu', strides=1)(embedded_sequences )
cnnouput=Dropout(0.2)(cnnouput)

    # personalised attention - word level
attention_a = Dot((2, 1))([cnnouput, Dense(400,activation='tanh')(user_embedding_word)])
attention_weight = Activation('softmax')(attention_a)
news_rep=keras.layers.Dot((1, 1))([cnnouput, attention_weight])
newsEncoder = Model([news_input,user_id], news_rep)

# browsing history as concatenation of MAX_SENTS articles
all_news_input = [keras.Input((MAX_SENT_LENGTH,), dtype='int32') for _ in range(MAX_SENTS)]
browsed_news_rep = [newsEncoder([news,user_id]) for news in all_news_input]
browsed_news_rep =concatenate([Lambda(lambda x: K.expand_dims(x,axis=1))(news) for news in browsed_news_rep],axis=1)

## user encoder
# personalised attention - article level
attention_news = keras.layers.Dot((2, 1))([browsed_news_rep, Dense(400,activation='tanh')(user_embedding_news)])
attention_weight_news = Activation('softmax')(attention_news)
user_rep=keras.layers.Dot((1, 1))([browsed_news_rep, attention_weight_news])

# candidate items - as pseudo K + 1 classification task
candidates = [keras.Input((MAX_SENT_LENGTH,), dtype='int32') for _ in range(1+npratio)]
candidate_vecs = [ newsEncoder([candidate,user_id]) for candidate in candidates]
logits = [keras.layers.dot([user_rep, candidate_vec], axes=-1) for candidate_vec in candidate_vecs]
logits = keras.layers.Activation(keras.activations.softmax)(keras.layers.concatenate(logits))

model = Model(candidates+all_news_input+[user_id], logits)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['acc'])


candidate_one = keras.Input((MAX_SENT_LENGTH,))
candidate_one_vec = newsEncoder([candidate_one,user_id])
score = keras.layers.Activation(keras.activations.sigmoid)(keras.layers.dot([user_rep, candidate_one_vec], axes=-1))
model_test = keras.Model([candidate_one]+all_news_input+[user_id], score)

for ep in range(3):
    traingen=generate_batch_data_train(all_train_pn,all_label,all_train_id, 100)
    model.fit_generator(traingen, epochs=1,steps_per_epoch=len(all_train_id)//100)
    testgen=generate_batch_data_test(all_test_pn,all_test_label,all_test_id, 100)
    click_score = model_test.predict_generator(testgen, steps=len(all_test_id)//100,verbose=1)
    from sklearn.metrics import roc_auc_score
    all_auc=[]
    all_mrr=[]
    all_ndcg=[]
    all_ndcg2=[]
    for m in all_test_index:
        if np.sum(all_test_label[m[0]:m[1]])!=0 and m[1]<len(click_score):
            all_auc.append(roc_auc_score(all_test_label[m[0]:m[1]],click_score[m[0]:m[1],0]))
            all_mrr.append(mrr_score(all_test_label[m[0]:m[1]],click_score[m[0]:m[1],0]))
            all_ndcg.append(ndcg_score(all_test_label[m[0]:m[1]],click_score[m[0]:m[1],0],k=5))
            all_ndcg2.append(ndcg_score(all_test_label[m[0]:m[1]],click_score[m[0]:m[1],0],k=10))
    results.append([np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2)])
    print(np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2))